Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 1.06 ms, sys: 4.3 ms, total: 5.36 ms
Wall time: 14.4 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.20.0.251/10419/1 Dashboard: http://10.20.0.251:8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.36 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 276 ms, sys: 24.4 ms, total: 301 ms
Wall time: 2.34 s


[1.6094202534551627,
 1.837005755417366,
 1.4026502388618747,
 1.438932286569523,
 1.1651002771836598,
 1.6730605538775163,
 1.2397100484926353,
 1.8625561690500918,
 1.4990968477253561,
 1.7030208915318998,
 1.2483843419326823,
 1.0693963679317118,
 1.2110526764280407,
 1.818455091290291,
 1.4467322244311804,
 1.682347663938093,
 1.4825556476508313,
 1.4230380783592822,
 1.2891092411059653,
 1.5560461274410344,
 1.6581180333877064,
 1.97848202960955,
 1.328334503543405,
 1.3352026879005694,
 1.7725962753317979,
 1.9327871463811914,
 1.617958326920268,
 1.3766009639578085,
 1.0908361227504366,
 1.884456733723488,
 1.6978606414268533,
 1.3293784853713517,
 1.3501974107163333,
 1.8585739296701311,
 1.4727131817486514,
 1.6682659853779374,
 1.7959600135959333,
 1.7299020835579877,
 1.81651651431264,
 1.577221588120031,
 1.0672512559727658,
 1.3854243367434198,
 1.5489927029933976,
 1.2398661216286917,
 1.7285014408200365,
 1.2956783393813842,
 1.1077365979943838,
 1.9973954949092656,
 1.8

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.8896582804569273,
 1.8965736254267198,
 1.6116245694353122,
 1.16291051509912,
 1.1880564200762507,
 1.022709273596416,
 1.9605102235148464,
 1.0807397494582607,
 1.5146695074892373,
 1.1965832990040222,
 1.7133368238585467,
 1.7736001586654746,
 1.9912582434563637,
 1.106220630652377,
 1.843735202006069,
 1.227662048876609,
 1.6901699603861933,
 1.9465194188576218,
 1.4060770683239014,
 1.806689006123013,
 1.4295327266370306,
 1.6898310764175355,
 1.7541890991445026,
 1.8743124014319354,
 1.8403615979013273,
 1.8716558535093175,
 1.6228231916766256,
 1.3712154058548078,
 1.735471306627864,
 1.3983824417007986,
 1.7201987853295637,
 1.3733386382989043,
 1.3131346605953436,
 1.6850606943638737,
 1.9926511503753064]

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 274 ms, sys: 17.5 ms, total: 291 ms
Wall time: 2.65 s
